This notebook was run on [kaggle](https://www.kaggle.com/).

In [1]:
!wget https://www.statmt.org/wmt12/un.es-en.tgz

--2025-03-04 19:00:08--  https://www.statmt.org/wmt12/un.es-en.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.32.28
Connecting to www.statmt.org (www.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1103180390 (1.0G) [application/x-gzip]
Saving to: ‘un.es-en.tgz’

un.es-en.tgz        100%[===================>]   1.03G   112MB/s    in 9.5s    

2025-03-04 19:00:18 (110 MB/s) - ‘un.es-en.tgz’ saved [1103180390/1103180390]



In [2]:
!tar -xvzf un.es-en.tgz

MultiUN4WMT12/README
MultiUN4WMT12/undoc.2000.es-en.en
MultiUN4WMT12/undoc.2000.es-en.es


In [3]:
!rm un.es-en.tgz

In [4]:
from typing import List
from tqdm import tqdm
import random
import json

In [5]:
random.seed(7)

In [6]:
train_f = 5e-4

In [7]:
ens = []
with open("/kaggle/working/MultiUN4WMT12/undoc.2000.es-en.en", "r") as f:
    while l := f.readline().strip().replace("\xa0", " "):
        ens.append(l)

In [8]:
en_last = ens[-1]
en_last

'Property transferred to other missions'

In [9]:
train_count = round(len(ens)*train_f)
train_count, len(ens)

(5598, 11196913)

In [10]:
ens_train = random.choices(ens, k=train_count)

In [11]:
del ens

In [12]:
ess = []
with open("/kaggle/working/MultiUN4WMT12/undoc.2000.es-en.es", "r") as f:
    while l := f.readline().encode().decode('utf-8').strip().replace("\xa0", " "):
        ess.append(l)

In [13]:
es_last = ess[-1]
es_last

'Bienes transferidos a otras misiones'

In [14]:
ess_train = random.choices(ess, k=train_count)

In [15]:
del ess

In [16]:
def preprocess_sentence(s: str, bos: str = "<bos>", eos: str = "<eos>") -> List[str]:
    return [bos] + list(s.replace("  ", " ")) + [eos]

In [17]:
preprocess_sentence(en_last), preprocess_sentence(es_last)

(['<bos>',
  'P',
  'r',
  'o',
  'p',
  'e',
  'r',
  't',
  'y',
  ' ',
  't',
  'r',
  'a',
  'n',
  's',
  'f',
  'e',
  'r',
  'r',
  'e',
  'd',
  ' ',
  't',
  'o',
  ' ',
  'o',
  't',
  'h',
  'e',
  'r',
  ' ',
  'm',
  'i',
  's',
  's',
  'i',
  'o',
  'n',
  's',
  '<eos>'],
 ['<bos>',
  'B',
  'i',
  'e',
  'n',
  'e',
  's',
  ' ',
  't',
  'r',
  'a',
  'n',
  's',
  'f',
  'e',
  'r',
  'i',
  'd',
  'o',
  's',
  ' ',
  'a',
  ' ',
  'o',
  't',
  'r',
  'a',
  's',
  ' ',
  'm',
  'i',
  's',
  'i',
  'o',
  'n',
  'e',
  's',
  '<eos>'])

In [18]:
len(ens_train), len(ess_train)

(5598, 5598)

In [19]:
dataset = ens_train + ess_train

In [20]:
del ens_train

In [21]:
del ess_train

In [22]:
len(dataset), dataset[0], dataset[-1]

(11196,
 'The Prisons Bill prohibits the use of solitary confinement except in the case of extremely violent offenders.',
 'La evidencia de estas dificultades ha suscitado también la necesidad de trabajar en el futuro en el desarrollo de un sistema de información como el que se ha aludido en los párrafos 128 a 132.')

In [23]:
def preprocess_dataset(dataset: List[str]):
    preprocessed_dataset = []
    for s in dataset:
        preprocessed_dataset.append(preprocess_sentence(s))
    return preprocessed_dataset

In [24]:
preprocessed_dataset = preprocess_dataset(dataset)

In [25]:
def bpe_train(dataset: List[str], num_target_rules: int = 32_000):
    vocab = set()
    rules = []
    
    for d in tqdm(dataset, desc="create initial vocab"):
        vocab = vocab.union(set(d))
    
    for _ in tqdm(range(num_target_rules), desc="train"):
        bigram_counter = {}
        for d in dataset:
            # print(d)
            for j in range(len(d)-1):
                unite = tuple(d[j:j+2])
                if unite in bigram_counter:
                    bigram_counter[unite] += 1
                else:
                    bigram_counter[unite] = 1

        mx_cnt = 0
        target = ()
        for unite, cnt in bigram_counter.items():
            if cnt > mx_cnt:
                mx_cnt = cnt
                target = unite
        # print(target_lst)

        vocab.add("".join(target))
        rules.append(target)

        # y = 0
        for i in range(len(dataset)):
            while True:
                d = dataset[i]
                left = -1
                for j in range(len(d)-1):
                    if d[j:j+2] == list(target):
                        left = j
                        break
                if left >= 0:
                    dataset[i] = d[:left] + [''.join(d[left:left+2])] + d[left+2:]
                else:
                    break
            # y += len(dataset[i])
        
    return vocab, rules

In [26]:
vocab, rules = bpe_train(preprocessed_dataset, num_target_rules=10_000)

train: 100%|██████████| 10000/10000 [1:26:42<00:00,  1.92it/s]


In [30]:
rules[:20]

[('e', ' '),
 ('s', ' '),
 ('n', ' '),
 ('a', ' '),
 ('o', ' '),
 ('e', 'r'),
 ('e', 'n'),
 ('o', 'n'),
 ('t', 'i'),
 ('c', 'i'),
 ('l', ' '),
 ('t', 'h'),
 ('r', 'e'),
 ('d', 'e '),
 ('a', 'r'),
 (',', ' '),
 ('a', 'n'),
 ('d', ' '),
 ('i', 'n'),
 ('o', 'r')]

In [31]:
len(rules), len(vocab)

(10000, 10146)

In [32]:
vocab_map = {e : i for i, e in enumerate(vocab)}

In [56]:
with open("bpe-en-es-10k-vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_map, f, ensure_ascii=False)

In [51]:
with open("bpe-en-es-10k-rules.txt", "w", encoding="utf-8") as f:
    for rule in rules:
        f.write(f"{vocab_map[rule[0]]} {vocab_map[rule[1]]}\n")

In [41]:
rules[1000:1010]

[('<bos>L', 'os '),
 ('al', 'es, '),
 ('el', 'y '),
 ('particul', 'ar '),
 ('200', '1'),
 ('ed ', 'by '),
 ('ad', 'ministr'),
 ('Or', 'ganiz'),
 ('us', 't '),
 ('e ', 'of the ')]

In [42]:
preprocessed_dataset[:1]

[['<bos>The ',
  'Pri',
  'sons ',
  'B',
  'ill ',
  'prohib',
  'its ',
  'the ',
  'use of ',
  'sol',
  'it',
  'ary ',
  'con',
  'fin',
  'ement ',
  'excep',
  't ',
  'in the ',
  'case of ',
  'extrem',
  'ely ',
  'viol',
  'ent ',
  'off',
  'ender',
  's.<eos>']]

In [43]:
def apply_bpe_rules(dataset, rules, preprocessed=False):
    if not preprocessed:
        p_dataset = preprocess_dataset(dataset)
    else:
        p_dataset = dataset.copy()

    for rule in rules:
        for i in range(len(p_dataset)):
            while True:
                d = p_dataset[i]
                left = -1
                for j in range(len(d)-1):
                    if d[j:j+2] == list(rule):
                        left = j
                        break
                if left >= 0:
                    p_dataset[i] = d[:left] + [''.join(d[left:left+2])] + d[left+2:]
                else:
                    break
    return p_dataset

In [44]:
test_dataset = [
    "Hello, where are you?",
    "Is this making any kind of sense?",
    "This is an imaginary situation.",
    "It is such a shame!"
]

pretokenized_test_dataset = apply_bpe_rules(test_dataset, rules)

In [45]:
pretokenized_test_dataset

[['<bos>H', 'ello, ', 'where ', 'are ', 'you', '?<eos>'],
 ['<bos>I',
  's ',
  'this ',
  'making ',
  'any ',
  'kind ',
  'of ',
  'sen',
  'se',
  '?<eos>'],
 ['<bos>This ', 'is ', 'an ', 'im', 'ag', 'in', 'ary ', 'situ', 'ation.<eos>'],
 ['<bos>It is ', 'such a ', 'sh', 'ame', '!', '<eos>']]

In [46]:
def tokenize_dataset(pre_tokenized_dataset, vocab):
    tokenized_dataset = []
    for d in pre_tokenized_dataset:
        tokenized_dataset.append(list(map(lambda tok: vocab[tok], d)))
    return tokenized_dataset

In [47]:
tokenized_test_dataset = tokenize_dataset(pretokenized_test_dataset, vocab_map)

In [48]:
tokenized_test_dataset

[[1254, 6565, 9845, 2964, 4346, 8849],
 [778, 4361, 3566, 2480, 1117, 8484, 8486, 4812, 2261, 8849],
 [7087, 3229, 7356, 2436, 4518, 3418, 2469, 6545, 7484],
 [2727, 7110, 9944, 10043, 7236, 3324]]